In [1]:
!pip install pandas timedelta


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from datetime import timedelta

# Load the Excel file
file_path = "./data/data.xlsx"
xls = pd.ExcelFile(file_path)

# Read all sheets into a dictionary of DataFrames
sheets = {sheet_name: xls.parse(sheet_name) for sheet_name in xls.sheet_names}

# Function to clean and process the dataset
def clean_transaction_data(df):
    """
    Data Cleaning & Validation:
    1. Identifies and removes duplicate records
    2. Excludes non-spending transactions
    3. Assigns start and end dates using a structured approach
    4. Handles missing values efficiently
    5. Implements error handling to prevent script failures
    6. Validates data consistency before finalizing
    """
    
    try:
        # Step 1: Remove Duplicate Transactions
        df['duplicate_flag'] = df.duplicated(subset=['date', 'line_amount_usd', 'memo'], keep=False)
        
        # Prioritize transactions from more reliable integrations (Bill.com > QuickBooks > Brex)
        integration_priority = {'bill.com': 3, 'quickbooks': 2, 'brex': 1}
        df['integration_score'] = df['integration'].map(integration_priority).fillna(0)
        df = df.sort_values(by=['duplicate_flag', 'integration_score'], ascending=[False, False])
        df = df.drop_duplicates(subset=['date', 'line_amount_usd', 'memo'], keep='first')
        
        # Step 2: Exclude Non-Spending Transactions
        df['Include/Exclude'] = df['record_type'].apply(lambda x: "Exclude" if x == "journal_entry" else "Include")
        
        # Step 3: Assign Start and End Dates
        for index, row in df.iterrows():
            if row['Include/Exclude'] == "Include":
                memo = str(row['memo']) if pd.notna(row['memo']) else ""
                
                if '-' in memo:
                    parts = memo.split('-')
                    try:
                        df.at[index, 'Start Date'] = pd.to_datetime(parts[0].strip(), errors='coerce')
                        df.at[index, 'End Date'] = pd.to_datetime(parts[1].strip(), errors='coerce')
                    except Exception:
                        df.at[index, 'Start Date'] = row['date']
                        df.at[index, 'End Date'] = row['date'] + timedelta(days=30)
                else:
                    df.at[index, 'Start Date'] = row['date']
                    df.at[index, 'End Date'] = row['date'] + timedelta(days=30)
            else:
                df.at[index, 'Start Date'] = None
                df.at[index, 'End Date'] = None
        
        # Step 4: Fill Missing Memos
        df['memo'].fillna("No Description Provided", inplace=True)
        
        # Step 5: Fill Missing Dates
        df['Start Date'].fillna(df['date'], inplace=True)
        df['End Date'].fillna(df['Start Date'] + timedelta(days=30), inplace=True)
        
        # Step 6: Fill Missing 'other_integration_list' Values
        if 'other_integration_list' in df.columns:
            df['other_integration_list'] = df['other_integration_list'].apply(lambda x: "No Other Integrations" if pd.isna(x) or x == "[]" else x)
        
        # Step 7: Add Notes for Justification
        df['Notes'] = df.apply(lambda row: 
            "Excluded due to duplicate or non-spending record." if row['Include/Exclude'] == "Exclude" else 
            "Included based on valid transaction and inferred service period.", axis=1)
        
        return df.drop(columns=['duplicate_flag', 'integration_score'])
    
    except Exception as e:
        print(f"Error in data cleaning process: {e}")
        return df

# Apply the cleaning function to all sheets
cleaned_sheets = {name: clean_transaction_data(df) for name, df in sheets.items()}

# Save the cleaned dataset to a new Excel file
output_file = "Cleaned_Data_QA_Specialist_Final.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for name, df in cleaned_sheets.items():
        df.to_excel(writer, sheet_name=name, index=False)

print(f"Data Cleaning Completed. Saved as: {output_file}")


Data Cleaning Completed. Saved as: Cleaned_Data_QA_Specialist_Final.xlsx
